In [1]:
!pip install nltk scikit-learn matplotlib tensorflow


Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 1.8 MB/s eta 0:00:00 0:00:010:00:01��━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 644.8/644.8 MB 189.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 2.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.2 MB/s eta 0:00:008.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 61.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 46.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 39.0 MB/s eta 0:00:00 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 46.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('stopwords')


2025-06-19 05:52:57.615783: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-19 05:52:57.740756: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-19 05:52:57.861761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750312377.973041     155 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750312378.005729     155 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750312378.221896     155 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

True

In [9]:
# Add encoding parameter to handle non-UTF-8 characters
df = pd.read_csv("SME-Training.csv", encoding='latin1')  # or try 'ISO-8859-1' or 'cp1252'
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']
df = df[['sentiment', 'text']]
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})  # Label mapping
df.drop_duplicates(inplace=True)


In [11]:
print ("*"*60)
print ("First 5 rows")
print ("*"*60)
df.head()         # First 5 rows

print ("*"*60)
print ("Last 5 rows")
print ("*"*60)
df.tail()         # Last 5 rows


************************************************************
First 5 rows
************************************************************
************************************************************
Last 5 rows
************************************************************


,sentiment,text
1599994,1,Just woke up. Having no school is the best fee...
1599995,1,TheWDB.com - Very cool to hear old Walt interv...
1599996,1,Are you ready for your MoJo Makeover? Ask me f...
1599997,1,Happy 38th Birthday to my boo of alll time!!! ...
1599998,1,happy #charitytuesday @theNSPCC @SparksCharity...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1583690 entries, 0 to 1599998
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1583690 non-null  int64 
 1   text       1583690 non-null  object
dtypes: int64(1), object(1)
memory usage: 36.2+ MB


In [13]:
df.shape

(1583690, 2)

In [14]:
df.columns

Index(['sentiment', 'text'], dtype='object')

In [15]:
df.describe(include='object')

,text
count,1583690
unique,1581465
top,@nikrichie aww why dont you go to bed and jus...
freq,2


In [17]:
df['sentiment'].value_counts()        # Frequency of each unique value

sentiment
1    793506
0    790184
Name: count, dtype: int64

In [18]:
df.isnull().sum()        # Count of missing values per column
df.duplicated().sum()    # Count of duplicated rows

0

In [19]:
df.dtypes     # Data types of each column


sentiment     int64
text         object
dtype: object

In [20]:
df.dropna(inplace=True)       # Drop rows with any missing values

In [22]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def clean_text(text):
    text = text.lower()                                # Lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Remove URLs
    text = re.sub(r'\@\w+|\#','', text)                # Remove mentions, hashtags
    text = re.sub(r"[^a-zA-Z\s]", '', text)            # Remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()           # Normalize whitespace
    stop_words = set(stopwords.words("english"))
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

df['clean_text'] = df['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/84bab8d2-1dc2-4c3e-849b-
[nltk_data]     b4acc5d8e636/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
X = df['clean_text']
y = df['sentiment']

vec = TfidfVectorizer(max_features=5000)
X_vec = vec.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

clf = MultinomialNB()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))


Accuracy: 0.7590500666165727
[[118310  38965]
 [ 37353 122110]]
              precision    recall  f1-score   support

           0       0.76      0.75      0.76    157275
           1       0.76      0.77      0.76    159463

    accuracy                           0.76    316738
   macro avg       0.76      0.76      0.76    316738
weighted avg       0.76      0.76      0.76    316738



In [24]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['clean_text'])

sequences = tokenizer.texts_to_sequences(df['clean_text'])
padded = pad_sequences(sequences, maxlen=100)

X_train, X_test, y_train, y_test = train_test_split(padded, y, test_size=0.2, random_state=42)


In [25]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test), batch_size=128)


/home/84bab8d2-1dc2-4c3e-849b-b4acc5d8e636/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-06-19 06:56:25.933094: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3


2025-06-19 06:56:29.733279: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 506780800 exceeds 10% of free system memory.


9899/9899 ━━━━━━━━━━━━━━━━━━━━ 53s 5ms/step - accuracy: 0.4999 - loss: 0.6932 - val_accuracy: 0.4965 - val_loss: 0.6932
Epoch 2/3
9899/9899 ━━━━━━━━━━━━━━━━━━━━ 54s 5ms/step - accuracy: 0.4990 - loss: 0.6932 - val_accuracy: 0.5035 - val_loss: 0.6931
Epoch 3/3
9899/9899 ━━━━━━━━━━━━━━━━━━━━ 52s 5ms/step - accuracy: 0.4989 - loss: 0.6932 - val_accuracy: 0.5035 - val_loss: 0.6931
